In [ ]:
from pathlib import Path
import mudata
import scanpy as sc
import scanpy.external as sce
import anndata as ad

Read and merge datasets

In [ ]:
runs = {}
for f in snakemake.input:
    name = Path(f).parent.parent.name
    runs[name] = mudata.read(f)["gene"]

adata = ad.concat(runs, join="inner", label="run")

Normalize

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

PCA

In [ ]:
sc.tl.pca(adata)

Integrate

In [ ]:
sce.pp.scanorama_integrate(adata, key="run")

Save

In [ ]:
adata.write_h5ad(snakemake.output[0], compression="gzip")